<h1> Importing Libraries    <h1>

In [1]:
import pandas as pd
import pandasql as ps
import psycopg2
import numpy as np
import json
from datetime import datetime
from datetime import timedelta

import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf
from ipywidgets import widgets
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()

<h1> Importing JSON file    <h1>

In [2]:
with open("/home/ys/notebook/config.json") as f:
    config = json.load(f)
print("Config:",config)
mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')


Config: {'database': 'Students', 'user': 'postgres', 'password': 'a', 'host': 'localhost', 'port': '5432'}


<h1> Get Time Stats and Saving them    <h1>

In [3]:
def get_abs_stats_by_time():
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    df =  pd.DataFrame(columns = ["date",'id_subject','nb_lates','nb_abs'] )
    id_time = []
    id_subject = []
    nb_lates = []
    nb_abs = []
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute("""
    select day,id_subject, 
        count(CASE WHEN is_late = true THEN 1 END) as nb_lates,
		count( CASE WHEN is_abs = true THEN 1 END) as nb_abs
    from "Students_db".student_abs join "Students_db".time
    on "Students_db".student_abs.id_time = "Students_db".time.id
    group by day,id_subject
    order by day;
    """)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        for j in range(len(myresult)) :
            id_time.append(myresult[j][0])
            id_subject.append(myresult[j][1])
            nb_lates.append(myresult[j][2])
            nb_abs.append(myresult[j][3])
        
        df['date'] = id_time
        df['id_subject'] = id_subject
        df['nb_lates'] = nb_lates
        df['nb_abs'] = nb_abs
        
        return df

In [4]:
df_time = get_abs_stats_by_time()
 
output = 'time_stats_list_output.csv'

df_time.to_csv(f"/home/ys/notebook/output/{output}",index=False)

df_time

,date,id_subject,nb_lates,nb_abs
0,2021-03-30,1,15,4
1,2021-04-06,1,16,36
2,2021-04-14,1,15,32
3,2021-04-20,1,24,38
4,2021-05-05,1,11,43


In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_time.date, y=df_time.nb_lates, mode = "lines+markers", name="Lates"))
fig.add_trace(go.Scatter(x = df_time.date, y=df_time.nb_abs,  mode = "lines+markers", name="Absents", line=dict(color='firebrick', width=3)))

fig.update_layout(title = "Total of Students Late/Absent by Time",title_x = 0.4, xaxis_title="Date", yaxis_title="Students count")
fig.update_layout(
    xaxis = dict(showline= True, showgrid=False, 
                   showticklabels=True, linecolor="grey", 
                  linewidth = 2, ticks="outside",
                  tickfont=dict(size=12, color="grey")
                  ),
    yaxis = dict(showline= True, showgrid=False, 
               showticklabels=True, linecolor="grey", 
              linewidth = 2, ticks="outside",
              tickfont=dict(size=12, color="grey")
              ),
    autosize = False,
    margin = dict(autoexpand=False, l=50, r=100, t=110),
    plot_bgcolor="white"
)

In [7]:
def get_abs_stats_by_fil():
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    df =  pd.DataFrame(columns = ["filiere",'id_subject','nb_lates','nb_abs'] )
    id_time = []
    id_subject = []
    nb_lates = []
    nb_abs = []
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute("""
    select filiere,id_subject, 
        count(CASE WHEN is_late = true THEN 1 END) as nb_lates,
		count( CASE WHEN is_abs = true THEN 1 END) as nb_abs
    from "Students_db".student_abs join "Students_db".student
    on "Students_db".student_abs.id_student = "Students_db".student.id
    group by filiere,id_subject
    order by filiere;
    """)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        for j in range(len(myresult)) :
            id_time.append(myresult[j][0])
            id_subject.append(myresult[j][1])
            nb_lates.append(myresult[j][2])
            nb_abs.append(myresult[j][3])
        
        df['filiere'] = id_time
        df['id_subject'] = id_subject
        df['nb_lates'] = nb_lates
        df['nb_abs'] = nb_abs
        
        return df

In [8]:
df_fil = get_abs_stats_by_fil()

output = 'filiere_stats_list_output.csv'

df_fil.to_csv(f"/home/ys/notebook/output/{output}",index=False)

df_fil

,filiere,id_subject,nb_lates,nb_abs
0,DS,1,29,72
1,DSE,1,52,81


In [10]:
fig = px.bar(
    df_fil, x="id_subject", y="nb_abs", text = "nb_abs", color="filiere",
             title = "Absents by filiere on Each Subject",
      labels = {"nb_abs" : "№ of students", "course": "Subject"},
      barmode= "group")
fig.update_traces(textposition="outside")
fig.update_layout(uniformtext_minsize = 4)
fig.update_layout(
xaxis=dict(showline= True, showgrid=False, 
           showticklabels=True, linecolor="grey", 
          linewidth = 2, ticks="outside",
          tickfont=dict(size=12, color="grey")),
yaxis=dict(showline= True, showgrid=False, 
           showticklabels=True, linecolor="grey", 
          linewidth = 2, ticks="outside",
          tickfont=dict(size=12, color="grey")),
    title_x = 0.4,
    autosize=False,
    margin=dict(
    autoexpand=False, l=50, r=100, t=50),
    plot_bgcolor="white"
)

In [12]:
fig = px.bar(
    df_fil, x="id_subject", y="nb_lates", text = "nb_lates", color="filiere",
             title = "Lates by filiere on Each Subject",
      labels = {"nb_lates" : "№ of students", "course": "Subject"},
      barmode= "group")
fig.update_traces(textposition="outside")
fig.update_layout(uniformtext_minsize = 4)
fig.update_layout(
xaxis=dict(showline= True, showgrid=False, 
           showticklabels=True, linecolor="grey", 
          linewidth = 2, ticks="outside",
          tickfont=dict(size=12, color="grey")),
yaxis=dict(showline= True, showgrid=False, 
           showticklabels=True, linecolor="grey", 
          linewidth = 2, ticks="outside",
          tickfont=dict(size=12, color="grey")),
    title_x = 0.4,
    autosize=False,
    margin=dict(
    autoexpand=False, l=50, r=100, t=50),
    plot_bgcolor="white"
)

<h1>  Get Stats about All Students and Saving them   <h1>

In [13]:
def get_all_student_names():
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    df =  pd.DataFrame(columns = ["id","first_name","last_name"] )
    ida = []
    fname = []
    lname = []
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT id,first_name,last_name FROM "Students_db".student ;""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        for j in range(len(myresult)) :
            ida.append((myresult[j][0]))
            fname.append((myresult[j][1]))
            lname.append((myresult[j][2]))
            
        df ["id"] = ida
        df ["first_name"] = fname
        df ["last_name"] = lname
        
        return df

def get_abs_stats_by_student():
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    df =  pd.DataFrame(columns = ["id_student",'id_subject','nb_lates','nb_abs','present'] )
    id_student = []
    id_subject = []
    nb_lates = []
    nb_abs = []
    present =[]
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute("""
    select id_student,id_subject,filiere,
		count(CASE WHEN is_late = true THEN 1 END) as nb_lates,
		count( CASE WHEN is_abs = true THEN 1 END) as nb_abs,
        count( CASE WHEN is_abs = false THEN 1 END) as present
    from "Students_db".student_abs join "Students_db".student
    on "Students_db".student.id = "Students_db".student_abs.id_student
    group by id_student,id_subject,filiere
    order by id_student
;
    """)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        for j in range(len(myresult)) :
            id_student.append(myresult[j][0])
            id_subject.append(myresult[j][1])
            nb_lates.append(myresult[j][2])
            nb_abs.append(myresult[j][3])
            present.append(myresult[j][4])
        
        df['id_student'] = id_student
        df['id_subject'] = id_subject
        df['nb_lates'] = nb_lates
        df['nb_abs'] = nb_abs
        df['present'] = present
        df_names = get_all_student_names()
        
        result = ps.sqldf("""
        
        select df_names.first_name, df_names.last_name , df.id_subject ,  df.nb_lates, df.nb_abs , df.present
        from df join df_names  
        where df_names.id = df.id_student
        """, locals())
        
        return result



In [14]:

df_students = get_abs_stats_by_student()
 
output = 'all_students_stats_list_output.csv'

df_students.to_csv(f"/home/ys/notebook/output/{output}",index=False)

df_students

,first_name,last_name,id_subject,nb_lates,nb_abs,present
0,Abdelakbir,MENANI,1,DS,0,4
1,Abdoul-Madjid,SANOUSSI LABO,1,DS,0,1
2,Amina,EZZOUINE,1,DS,1,0
3,Amine,AJARRAY,1,DS,2,2
4,Amine,CHEDDAD,1,DS,0,5
...,...,...,...,...,...,...
74,Mahamadou,SANGARE,1,DSE,1,0
75,El mehdi,SOUMMER,1,DSE,0,3
76,Oussama,YOUSR,1,DSE,4,0
77,Soukaina,ZARROUQ,1,DSE,0,4


<h1> Get Stats about Students of 'filieres' and Saving them    <h1>

In [15]:
def get_abs_stats_by_student_fil(fil):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    df =  pd.DataFrame(columns = ["id_student",'id_subject','nb_lates','nb_abs','present'] )
    id_student = []
    id_subject = []
    nb_lates = []
    nb_abs = []
    present = []
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""
   select id_student,id_subject,
		count(CASE WHEN is_late = true THEN 1 END) as nb_lates,
		count( CASE WHEN is_abs = true THEN 1 END) as nb_abs,
        count( CASE WHEN is_abs = false THEN 1 END) as present
    from "Students_db".student_abs
    where id_student in (select id from "Students_db".student where filiere = '{fil}')
    group by id_student,id_subject
    order by id_student
;

    """)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        for j in range(len(myresult)) :
            id_student.append(myresult[j][0])
            id_subject.append(myresult[j][1])
            nb_lates.append(myresult[j][2])
            nb_abs.append(myresult[j][3])
            present.append(myresult[j][4])
        df['id_student'] = id_student
        df['id_subject'] = id_subject
        df['nb_lates'] = nb_lates
        df['nb_abs'] = nb_abs
        df['present'] = present
        
        df_names = get_all_student_names()
        
        result = ps.sqldf("""
        
        select df_names.first_name, df_names.last_name , df.id_subject ,  df.nb_lates, df.nb_abs ,df.present
        from df join df_names  
        where df_names.id = df.id_student
        """, locals())
        
        return result


In [16]:
fil = 'DSE'
df_students_fil = get_abs_stats_by_student_fil(fil)
output = fil+'_students_stats_list_output.csv'
df_students_fil.to_csv(f"/home/ys/notebook/output/{output}",index=False)

df_students_fil

,first_name,last_name,id_subject,nb_lates,nb_abs,present
0,Yassine,AIT MALEK,1,0,4,1
1,Younes,AIT M'HA,1,1,4,1
2,Salah-eddine,AOUARI,1,1,1,4
3,Soufyane,ATICHE,1,1,3,2
4,Chaimaa,ATRAOUI,1,2,1,4
5,Achraf,BALIJ,1,1,0,5
6,Fathiya,BANAH,1,0,0,5
7,Hajar,BOUDINAR,1,1,0,5
8,Sabah,BOUKILI,1,0,1,4
9,Hibatallah,BOUNAR,1,0,0,5


In [17]:
fil = 'DS'
df_students_fil = get_abs_stats_by_student_fil(fil)
output = fil+'_students_stats_list_output.csv'
df_students_fil.to_csv(f"/home/ys/notebook/output/{output}",index=False)

df_students_fil

,first_name,last_name,id_subject,nb_lates,nb_abs,present
0,Abdelakbir,MENANI,1,0,4,1
1,Abdoul-Madjid,SANOUSSI LABO,1,0,1,4
2,Amina,EZZOUINE,1,1,0,5
3,Amine,AJARRAY,1,2,2,3
4,Amine,CHEDDAD,1,0,5,0
5,Amine,ELOMAIRI,1,0,5,0
6,Andre,SINARE,1,0,1,4
7,Asma,OUSSAR,1,2,2,3
8,Aymane,HAMDI,1,0,5,0
9,Ayoub,ELHADINE,1,2,2,3
